In [72]:
from sentence_transformers import SentenceTransformer
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import torch
import json
import re
device = "cuda" if torch.cuda.is_available() else "cpu"

In [73]:
def process_input_json(filepath: str, jobs: dict, divisions: dict):
    with open(filepath) as f: data = json.load(f)
    for key in data:
        if key.startswith("coopJob_") and data[key]["jobId"] not in jobs: jobs[data[key]["jobId"]] = data[key]
        elif key.startswith("division_"): divisions[int(key.split("_")[1])] = data[key]

In [76]:
with open("ww_data_2.json") as f: data = json.load(f)

In [116]:
job_columns = {
    "job_id": pd.Series(dtype="int"), # key
    "division_id": pd.Series(dtype="int"), # 
    "application_deadline": pd.Series(dtype="datetime64[ns]"), # jobs[key]["postingListData"]["deadline"]
    "application_documents_required": pd.Series(dtype="object"),  # jobs[key]["pageData"]["Application Information"]["Application Documents Required"]
    "company": pd.Series(dtype="str"), # jobs[key]["postingListData"]["company"]
    "division": pd.Series(dtype="str"), # jobs[key]["postingListData"]["division"]
    "min_salary": pd.Series(dtype="float"), # jobs[key]["pageData"]["Job Posting Information"]["Compensation and Benefits"]
    "max_salary": pd.Series(dtype="float"), # jobs[key]["pageData"]["Job Posting Information"]["Compensation and Benefits"]
    "compensation_and_benefits": pd.Series(dtype="str"), # jobs[key]["pageData"]["Job Posting Information"]["Compensation and Benefits"]
    "work_arrangement": pd.Series(dtype="str"), # jobs[key]["pageData"]["Job Posting Information"]["Employment Location Arrangement"]
    "city": pd.Series(dtype="str"), # jobs[key]["pageData"]["Job Posting Information"]["Job - City"]
    "province": pd.Series(dtype="str"), # jobs[key]["pageData"]["Job Posting Information"]["Job - Province/State"]
    "country": pd.Series(dtype="str"), # jobs[key]["pageData"]["Job Posting Information"]["Job - Country"]
    "job_responsibilities": pd.Series(dtype="str"), # jobs[key]["pageData"]["Job Posting Information"]["Job Responsibilities"]
    "job_summary": pd.Series(dtype="str"), # jobs[key]["pageData"]["Job Posting Information"]["Job Summary"]
    "job_title": pd.Series(dtype="str"), # jobs[key]["postingListData"]["jobTitle"]
    "required_skills": pd.Series(dtype="str"), # jobs[key]["pageData"]["Job Posting Information"]["Required Skills"]
    "duration": pd.Series(dtype="int"), # jobs[key]["postingListData"]["Work Term Duration"]
    "num_job_openings": pd.Series(dtype="int"), # jobs[key]["postingListData"]["openings"]
    "num_applications": pd.Series(dtype="int"), # jobs[key]["postingListData"]["applications"]
}
jobs_df = pd.DataFrame(data=[], columns=job_columns.keys())

In [121]:
data["division_10183"]

{'division': 'Divisional Office',
 'graphs': [{'series': [{'colorByPoint': True,
     'data': [{'name': 'Arts', 'y': 55},
      {'name': 'Engineering', 'y': 21},
      {'name': 'Health', 'y': 3},
      {'name': 'Mathematics', 'y': 16},
      {'name': 'Science', 'y': 5}],
     'name': 'Percentage'}],
   'title': 'Hires by Faculty'},
  {'series': [{'colorByPoint': True,
     'data': [{'name': 'First', 'y': 34},
      {'name': 'Second', 'y': 18},
      {'name': 'Third', 'y': 18},
      {'name': 'Fourth', 'y': 21},
      {'name': 'Fifth', 'y': 5},
      {'name': 'Sixth +', 'y': 3}],
     'name': 'Percentage'}],
   'title': 'Hires by Student Work Term Number'},
  {'categories': ['Psychology',
    'Computer Engineering',
    'Liberal Studies',
    'Computer Science/BCS',
    'Economics',
    'Legal Studies',
    'Political Science',
    'Systems Design Engineering',
    'Financial Analysis & Risk Management',
    'Nanotechnology Engineering'],
   'series': [{'data': [7, 5, 5, 4, 4, 2, 2, 2, 

In [ ]:
division_columns = {
    "hiring_history": pd.Series(dtype="list[int]"), 
    "hires_by_faculty": pd.Series(dtype="dict[str, int]"), 
    "hires_by_work_term_number": pd.Series(dtype="dict[int, int]"), 
    "most_frequently_hired_programs": pd.Series(dtype="dict[str, int]"), 
    "work_term_ratings": pd.Series(dtype="int"), 
    "availability_of_employer_support": pd.Series(dtype="float"), 
    "opportunities_to_learn_or_develop_new_skills": pd.Series(dtype="float"), 
    "opportunities_to_make_meaningful_contributions_at_work": pd.Series(dtype="float"), 
    "opportunities_to_expand_your_professional_network": pd.Series(dtype="float"), 
    "appropriate_compensation_and_or_benefits": pd.Series(dtype="float"), 
    "how_closely_your_work_was_related_to_your_academic_program": pd.Series(dtype="float"), 
    "how_closely_your_work_was_related_to_the_skills_you_are_developing_at_university": pd.Series(dtype="float")
}
divisions_df = pd.DataFrame(data=[], columns=division_columns.keys())

1) 

In [117]:
filepaths = ["ww_data_1.json", "ww_data_2.json"]
jobs = {}
divisions = {}
for filepath in filepaths: process_input_json(filepath, jobs, divisions)

In [ ]:
job_descriptions = {
    job_id:
    re.sub(
        r"\n\n+",
        "\n",
        BeautifulSoup(
            f"\
Job Title: {job['pageData']['Job Posting Information']['Job Title']}\n\
Company: {job['pageData']['Company Information']['Organization']} - {job['pageData']['Company Information']['Division']}\n\
Job Summary: {job['pageData']['Job Posting Information']['Job Summary']}\n\
Job Responsibilities: {job['pageData']['Job Posting Information']['Job Responsibilities']}\n\
Required Skills: {job['pageData']['Job Posting Information']['Required Skills']}",
            "html.parser"
        ).get_text()
    )
    for job_id, job in jobs.items()
}

In [ ]:
model = SentenceTransformer('nomic-ai/nomic-embed-text-v1', trust_remote_code=True).to(device)

A new version of the following files was downloaded from https://huggingface.co/nomic-ai/nomic-bert-2048:
- configuration_hf_nomic_bert.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/nomic-ai/nomic-bert-2048:
- modeling_hf_nomic_bert.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
!!!!!!!!!!!!megablocks not available, using torch.matmul instead
C:\Users\parth\.cache\huggingface\modules\transformers_modules\nomic-ai\nomic-bert-2048\359596ab182dcf943b7ca9e3f8809b6c2eaf652f\modeling_hf_nomic_bert.py:116: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which 

In [ ]:
columns = {
    "Job ID": pd.Series(dtype="int"),
    "Is For My Program": pd.Series(dtype="bool"), 
    "Job Title": pd.Series(dtype="str"), 
    "Job Summary": pd.Series(dtype="str"), 
    "Job Responsibilities": pd.Series(dtype="str"), 
    "Required Skills": pd.Series(dtype="str"), 
    "Min Salary": pd.Series(dtype="float"), 
    "Max Salary": pd.Series(dtype="float"), 
    "Location": pd.Series(dtype="str"), 
    "Work Arrangement": pd.Series(dtype="str"), 
    "Duration": pd.Series(dtype="int"), 
    "Applications": pd.Series(dtype="int"), 
    "Hiring History": pd.Series(dtype="list[int]"), 
    "Hires by Faculty": pd.Series(dtype="dict[str, int]"), 
    "Hires by Work Term Number": pd.Series(dtype="dict[int, int]"), 
    "Most Frequently Hired Programs": pd.Series(dtype="dict[str, int]"), 
    "Work Term Ratings": pd.Series(dtype="int"), 
    "Availability of employer support": pd.Series(dtype="float"), 
    "Opportunities to learn or develop new skills": pd.Series(dtype="float"), 
    "Opportunities to make meaningful contributions at work": pd.Series(dtype="float"), 
    "Opportunities to expand your professional network": pd.Series(dtype="float"), 
    "Appropriate compensation and/or benefits": pd.Series(dtype="float"), 
    "How closely your work was related to your academic program": pd.Series(dtype="float"), 
    "How closely your work was related to the skills you are developing at university": pd.Series(dtype="float")
}
jobs_df = pd.DataFrame(data=[], columns=columns.keys())